# Summer 2025 Applied NLP Homework 4 (Part 2 of 2)

## Instructors: Dr. Mahdi Roozbahani, Wafa Louhichi, Dr. Nimisha Roy

## Deadline: July 25th, 11:59PM AoE

## Honor Code and Assignment Deadline
<!-- No changes needed on the below section -->
* No unapproved extension of the deadline is allowed. Late submission will lead to 0 credit. 

* Discussion is encouraged on Ed as part of the Q/A. However, all assignments should be done individually.

* <font color='darkred'>Plagiarism is a **serious offense**. You are responsible for completing your own work. You are not allowed to copy and paste, or paraphrase, or submit materials created or published by others, as if you created the materials. All materials submitted must be your own.</font>

* <font color='darkred'>All incidents of suspected dishonesty, plagiarism, or violations of the Georgia Tech Honor Code will be subject to the institute’s Academic Integrity procedures. If we observe any (even small) similarities/plagiarisms detected by Gradescope or our TAs, **WE WILL DIRECTLY REPORT ALL CASES TO OSI**, which may, unfortunately, lead to a very harsh outcome. **Consequences can be severe, e.g., academic probation or dismissal, grade penalties, a 0 grade for assignments concerned, and prohibition from withdrawing from the class.**
</font>


## Instructions for the assignment 

<!-- No changes needed on the below section -->
- This entire assignment will be autograded through Gradescope:
    - **REQUIRED - Homework 4 Q7**: Submit the Q7 files to this section
    - **Bonus - Homework 4 Q8**: Submit the Q8 files to this section
    - **See HW4 Part 1 for Q1-Q5 (Required) and Q6 (Bonus)**


- We provided you different .py files and we added libraries in those files please DO NOT remove those lines and add your code after those lines. Note that these are the only allowed libraries that you can use for the homework.

- You will submit your implemented .py files to the corresponding homework section on Gradescope. 

# Google Colab Setup (Optional for running on Colab)
If you choose to work on the assignment on Google Colab, the following cell may help get you set up. You may need to right click on the Applied NLP folder and `Add shortcut to Drive`. You do not have to run this cell if you are working on the notebook locally.

In [21]:
# # Mount google drive
# from google.colab import drive
# drive.mount('/content/drive/')
# # You may need to create an Applied_NLP/HW#/hw#_code/ folder
# %cd '/content/drive/MyDrive/Applied_NLP/HW#/hw#_code/'

# ## This wraps output text according to the window size
# from IPython.display import HTML, display

# def set_css():
#   display(HTML('''
#   <style>
#     pre {
#         white-space: pre-wrap;
#     }
#   </style>
#   '''))
# get_ipython().events.register('pre_run_cell', set_css)

# Assignment Overview

In this homework, we will explore a RAG Pipeline for the task of answering multiple choice questions about NLP research papers. In our RAG pipeline, we will use FAISS as our vector store and HuggingFace for our models. You may explore different models to see what best fits this task, but for autograding purposes, Gradescope will automatically assign the LLM model for grading consistency. 

As a bonus exercise, you will deploy the Q7 RAG pipeline using Ollama and Flask instead of HuggingFace. Question 8 will need to be implemented locally, it is not compatible with Google Colab.

## Deliverables and Points Distribution

### Q7: RAG Pipeline [12.5pts]
- **7.1 Implementing the Retriever** [2pts] Deliverables: <font color = 'green'>retriever.py</font>

    - [1.5pt] loadDocuments

    - [1.5pt] splitDocuments

    - [3pt] createRetriever
- **7.2 Implementing the RAG Chain** [2.5pts] Deliverables: <font color = 'green'>rag_chain.py</font>

    - \_\_init\_\_

    - init_retriever_system

    - [2pt] createPrompt

    - [4.5pts] createRAGChain
 
### Q8: Deploy RAG and LLM Using Ollama and Flask [5.5pts Bonus Extra Credit] 
- **8.1 Using Ollama with RAG** [0pts] Deliverables: <font color = 'green'>rag_chain.py</font>

    - [0pt] set_ollama_only

- **8.2 Deploying an LLM and using it with RAG** [0pts] Deliverables: <font color = 'green'>llm_app.py and rag_chain.py</font>

    - [0pt] generate

    - [0pts] set_flask_ollama
 
- **8.3 Deploying RAG using FlaskG** [0pts] Deliverables: <font color = 'green'>rag_app.py and hashed_answers.json</font>

    - [0pt] query

    - [5.5pts] running ta_pipeline to get hashed_answers.json

# Setup
**Please checkout the environment_setup.md file to create the environment for this homework.** This notebook is tested under the package versions noted in the Library Imports cell output below, and the corresponding packages can be downloaded from [miniconda](https://docs.conda.io/en/latest/miniconda.html). 

In the .py files please implement the functions that have `raise NotImplementedError`, and after you finish the coding, please delete or comment out `raise NotImplementedError`.

## Library Imports

In [22]:
# # If using Google Colab, uncomment this cell and run
# !pip install numpy==1.26
# !pip install transformers==4.44
# !pip install ollama
# !pip install langchain==0.2.14
# !pip install langchain-community==0.2.12
# !pip install sentence-transformers==3.0.1
# !pip install faiss-cpu==1.8.0.post1
# !pip install pypdf==4.3.1
# !pip install python-dotenv
# !pip install langchain-openai langchain-core==0.2.41
# !pip install gradio_client

In [1]:
#Import the necessary libraries

import numpy as np
import sys
import langchain
import langchain_community
import langchain_core
import langchain_openai
import sentence_transformers
import pypdf
import dotenv
import importlib.metadata

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

print('Version information')

print('python: {}'.format(sys.version))
print('numpy: {}'.format(np.__version__))
print('langchain: {}'.format(langchain.__version__))
print('langchain_community: {}'.format(langchain_community.__version__))
print('sentence_transformers: {}'.format(sentence_transformers.__version__))
print('pypdf: {}'.format(pypdf.__version__))
print('langchain_core: {}'.format(langchain_core.__version__))
print('langchain_openai: {}'.format(importlib.metadata.version("langchain-openai")))
print('python-dotenv: {}'.format(importlib.metadata.version("python-dotenv")))
print('ollama: {}'.format(importlib.metadata.version("ollama")))

/Users/rutomo/opt/anaconda3/envs/nlp_hw4_gai_summer_2025/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Version information
python: 3.10.18 (main, Jun  5 2025, 08:13:51) [Clang 14.0.6 ]
numpy: 1.24.3
langchain: 0.2.14
langchain_community: 0.2.12
sentence_transformers: 3.0.1
pypdf: 4.3.1
langchain_core: 0.2.43
langchain_openai: 0.1.25
python-dotenv: 1.1.1
ollama: 0.5.1


# Q7: Introduction to Retrieval Augmented Generation (RAG) [4.5pts]

State-of-the-art Large Language Models (LLM's) are trained on an enormous corpus of textual data (taken from webpages, articles, books etc.), and they store a wide range of general knowledge in their parameters. While they are able to perform well on tasks that require general knowledge, they tend to struggle on tasks that require information that wasn't present in the training data. For instance, LLM's may struggle on tasks that require knowledge of domain-specific information, or even up-to-date information. 

It is very important to overcome this problem, because it is undesirable to get a non-answer from the LLM, and potentially even dangerous if the LLM begins to hallucinate (i.e ramble on with an answer that seems believable but is factually inaccurate). Therefore, it is essential to bridge the gap between the LLM's general knowledge, and other domain-specific or up-to-date information in order to help the LLM generate responses that are contextual and factually accurate, while reducing the chances of hallucinations.

There are two effective ways of accomplishing this:
1. <strong>Fine-tuning the LLM on the domain-specific/proprietary/new data</strong>: 
    <ul>
        <li>By fine-tuning the model, it can be made suitable for the task.</li>
        <li>However, this comes with some limitations. It is compute-intensive, expensive and not agile (it's not realistic to fine-tune an LLM with the new data coming in everyday)</li>
    </ul>
2. <strong>Retrieval Augmented Generation (RAG)</strong>: 
    <ul>
        <li><a href='https://arxiv.org/abs/2005.11401'>This technique</a> provides the LLM with contextual information from an external knowledge source that can be updated more easily.</li>
        <li>It allows the LLM to generate more contextual, and factually accurate responses by allowing it to dynamically access information from an external knowledge source.</li>
    </ul> 

<center><img src="data/images/rag-architecture.png" alt="drawing" width="700" align='center'></center>
<center>Basic RAG Architecture</center>


The RAG pipeline consists of the following steps:
<ol>
    <li><strong>Retrieval</strong>: The user's query is used to retrieve the relevant contextual information from the external knowledge source. The external knowledge source is a vector store that contains the embeddings of the documents that contain the proprietary/domain-specific data. The user query is embedded into the same vector space as these documents, and a similarity search is performed in this embedding space to retrieve the documents that are most similar to the user's query. These retrieved documents make up the context that the LLM will consider in order to produce factually accurate responses. </li>
    <li><strong>Augmentation</strong>: The user's query is augmented with the retrieved contextual information to form the prompt. The prompt will also usually include instructions to the LLM for performing the task. There is an entire sub-field known as Prompt Engineering, that is dedicated to fine-tuning the prompt so as to get the best possible response from the LLM, and you will get some experience with it in <strong>7.2</strong> while implementing the RAG Chain. </li>
    <li><strong>Generation</strong>: The constructed prompt, including the instructions to the LLM, the user query and the retrieved context, is fed to the LLM to generate a response that is comprehensible and factually correct.</li>
</ol>

## 7.1 : Implementing the Retriever [3pts]


In this section we will implement the Retriever that will be used in the RAG Chain. First we will load the documents to be used as the basis of the external knowledge source. Then we will split the documents into smaller chunks to be used with the vector database. Finally we will create the Retriever object that will be used with our RAG Chain to retrieve relevant document chunks as additional contextual information.

### 7.1.1 - 7.1.2 : Loading and Pre-processing data: Document Loaders and Text Splitters [1.5pts each]

Langchain provides <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/'>several classes</a> to load data into Document objects. There are classes to load data from HTML files, PDF files, JSON files, CSV files, file directories etc. In this homework, we'll be using Langchain's <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#pypdf-directory'>PyPDFDirectoryLoader</a> to load PDF's from a directory. A Document object is a dictionary that stores the text and metadata about each document.

Once the data has been loaded into Document(s), it's common to split the documents into smaller chunks. This is done because when a user inputs a query to the system, the retriever will return the most relevant documents, which will be augmented to the prompt that is sent to the LLM. There are limits to the length of this prompt, since it must fit into the LLM's context window. Therefore, it's common to split documents into smaller chunks, so that only the retrieved relevant chunks are inserted into the prompt. Langchain offers several <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/'>TextSplitters</a> to perform the document splitting. In this homework, we use the RecursiveCharacterTextSplitter which is a way to split a document into several chunks in a manner such that related pieces of text are kept together in a chunk.

In the **retriever.py** file complete the following functions:

* **loadDocuments**
* **splitDocuments**

In [2]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

from retriever import Retriever
from local_tests.retriever_test import Retriever_Test

local_test = Retriever_Test()
stu_retriever = Retriever()
num_chunks_to_query = 2

print('Local Tests for Loading and Splitting Documents \n')

# Local test for load documents
output_documents = stu_retriever.loadDocuments(data_dir='./data/papers/')
load_test = (len(output_documents) == local_test.load_documents_len)
print('Your load documents works as expected:', load_test)

# Local test for split documents
output_chunks = stu_retriever.splitDocuments(output_documents, chunk_size=700, chunk_overlap=50)
split_test = (len(output_chunks) == local_test.split_documents_len)
print('Your split documents works as expected:', split_test)

Local Tests for Loading and Splitting Documents 

Your load documents works as expected: True
Your split documents works as expected: True


In [3]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

# Load the PDF documents
from retriever import Retriever
retriever = Retriever()
documents = retriever.loadDocuments(data_dir='./data/papers/')

print('documents type:', type(documents)) # python list
print('documents[0] type:', type(documents[0]))   # Document object
print('documents length:', len(documents))   # each page is loaded as a separate document (104 pages -> 104 documents)

print('\nContent of documents[0]:\n', documents[0])   # the first document object, containing 'page_content' and 'metadata' fields

documents type: <class 'list'>
documents[0] type: <class 'langchain_core.documents.base.Document'>
documents length: 104

Content of documents[0]:
 page_content='Improving Language Understanding
by Generative Pre-Training
Alec Radford
OpenAI
alec@openai.comKarthik Narasimhan
OpenAI
karthikn@openai.comTim Salimans
OpenAI
tim@openai.comIlya Sutskever
OpenAI
ilyasu@openai.com
Abstract
Natural language understanding comprises a wide range of diverse tasks such
as textual entailment, question answering, semantic similarity assessment, and
document classiﬁcation. Although large unlabeled text corpora are abundant,
labeled data for learning these speciﬁc tasks is scarce, making it challenging for
discriminatively trained models to perform adequately. We demonstrate that large
gains on these tasks can be realized by generative pre-training of a language model
on a diverse corpus of unlabeled text, followed by discriminative ﬁne-tuning on each
speciﬁc task. In contrast to previous approaches, w

In [4]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

# Split the documents into smaller chunks
document_chunks = retriever.splitDocuments(documents)

print('document_chunks type:', type(document_chunks)) # python list
print('document_chunks[0] type:', type(document_chunks[0])) # Document object
print('document_chunks length:', len(document_chunks)) # you should observe that each document (corresponding to a page in a PDF) has been split into several chunks

for chunk in document_chunks[:3]:   # displaying the first 3 chunks
    print('\n\n', chunk)

document_chunks type: <class 'list'>
document_chunks[0] type: <class 'langchain_core.documents.base.Document'>
document_chunks length: 701


 page_content='Improving Language Understanding
by Generative Pre-Training
Alec Radford
OpenAI
alec@openai.comKarthik Narasimhan
OpenAI
karthikn@openai.comTim Salimans
OpenAI
tim@openai.comIlya Sutskever
OpenAI
ilyasu@openai.com
Abstract
Natural language understanding comprises a wide range of diverse tasks such
as textual entailment, question answering, semantic similarity assessment, and
document classiﬁcation. Although large unlabeled text corpora are abundant,
labeled data for learning these speciﬁc tasks is scarce, making it challenging for
discriminatively trained models to perform adequately. We demonstrate that large' metadata={'source': 'data/papers/gpt.pdf', 'page': 0}


 page_content='gains on these tasks can be realized by generative pre-training of a language model
on a diverse corpus of unlabeled text, followed by discriminative ﬁne-

In [5]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

# analysis of chunk size
def compute_avg_chunk_size(chunks):
    return sum([len(chunk.page_content) for chunk in chunks])/len(chunks)

print(f'Before split, there were {len(documents)} documents, with average size equal to {compute_avg_chunk_size(documents)}.')
print(f'After split, there were {len(document_chunks)} documents (chunks), with average size equal to {compute_avg_chunk_size(document_chunks)}.')

Before split, there were 104 documents, with average size equal to 4149.2307692307695.
After split, there were 701 documents (chunks), with average size equal to 624.5235378031384.


### 7.1.3 : Creating the Vector Database and Retrieval System: Embedding models and Vector Stores [3pts]

<a href='https://python.langchain.com/v0.2/docs/integrations/text_embedding/'>Langchain provides several embedding models</a>. An embedding model converts a piece of natural language (e.g. a token) into a vector embedding. You should have some knowledge of word embedding models such as Word2Vec and GloVe. We will be using Huggingface's BGE Embedding models, which are the one of the best open-source embedding models <a href='https://python.langchain.com/v0.2/docs/integrations/text_embedding/bge_huggingface/'>(according to Langchain)</a>. The model we use (BAAI/bge-small-en-v1.5) has 384-dimensional embedding vectors.

Once we have our embedding model, we need to create the vectorstore by embedding all of the document chunks into the vector space. A retriever will be used to retrieve the document chunks from the vectorstore that are most similar to the user's query using efficient similarity search algorithms. Langchain offers <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/'>several vectorstores</a>, and we will be using Facebook's AI Similarity Search (FAISS). 

In the **retriever.py** file complete the following functions:

* **createRetriever**

In [6]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

from retriever import Retriever
from local_tests.retriever_test import Retriever_Test

local_test = Retriever_Test()
stu_retriever = Retriever()
num_chunks_to_query = 2

print('Local Tests for Retriever \n')   # ensure that the local tests for the loading and splitting documents pass

output_documents = stu_retriever.loadDocuments(data_dir='./data/papers/')
output_chunks = stu_retriever.splitDocuments(output_documents, chunk_size=700, chunk_overlap=50)

# Local test for retriever embeddings
output_retrieval_system = stu_retriever.createRetriever(output_chunks, num_chunks_to_return=num_chunks_to_query)
student_embedding_model = stu_retriever.huggingface_embeddings
output_embedding = np.array(student_embedding_model.embed_query(output_chunks[0].page_content))
embedding_shape = output_embedding.shape[0]

if (student_embedding_model.model_name == local_test.model_name): 
    embedding_shape_test = (embedding_shape == local_test.embedding_size)
    print('Your retriever embeddings returns the expected shape:', embedding_shape_test)
else:
    print('You are free to choose the embedding model to use for the best results (provided it works with Gradescope),')
    print(f'but we can only locally test the embedding shape (no value testing) if using the {local_test.model_name} model')
    print('and the RecursiveCharacterTextSplitter.')

# Local test for chunk relevance
output_retrieved_chunks = output_retrieval_system.invoke(local_test.relevance_prompt)
returned_count_test = (len(output_retrieved_chunks) == num_chunks_to_query)
relevance_test = True
for chunk in output_retrieved_chunks:
    if local_test.relevant_pdf not in chunk.metadata['source']:
        relevance_test = False
print('Your retriever returns the expected number of chunks:', returned_count_test)
print('Your retriever returns chunks from the expected relevant documents:', relevance_test)

Local Tests for Retriever 

Your retriever embeddings returns the expected shape: True
Your retriever returns the expected number of chunks: True
Your retriever returns chunks from the expected relevant documents: True


In [7]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

# create the retriever
retrieval_system = retriever.createRetriever(document_chunks)
embedding_model = retriever.huggingface_embeddings

In [8]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

# sample embedding for a document chunk
sample_embedding = np.array(embedding_model.embed_query(document_chunks[0].page_content))
print("Size of the embedding: ", sample_embedding.shape)
print("Sample embedding of a document chunk: ", sample_embedding)

Size of the embedding:  (384,)
Sample embedding of a document chunk:  [-1.86509583e-02 -1.27523541e-02 -4.42757010e-02  1.99839976e-02
  4.54358570e-02 -8.86677764e-03  6.96528179e-04  1.14407567e-02
  3.35411355e-02 -3.01743709e-02 -2.33880375e-02 -6.56098872e-02
  6.75567240e-02  7.66655132e-02  6.66777343e-02  2.27699522e-02
 -4.07686736e-03  2.88760904e-02 -1.48760751e-02 -1.81919839e-02
  2.06040591e-02 -5.04119545e-02  6.91428606e-04 -1.10212546e-02
 -5.39466664e-02  4.52645356e-03  3.32243694e-03 -3.94856669e-02
 -1.03674065e-02 -2.04585850e-01  2.24794075e-02  2.93982495e-02
  8.67721438e-02  6.67661009e-03  6.90001808e-03  3.32516544e-02
 -3.38732228e-02  1.15591940e-02  2.11533215e-02 -1.29593629e-02
 -1.53322490e-02 -1.74208246e-02 -5.63682523e-03 -4.73322459e-02
  1.91294141e-02 -2.30338313e-02 -1.88581925e-02 -4.31183949e-02
 -7.51457363e-02 -2.49210652e-02 -5.70337102e-02 -8.07441622e-02
  1.16802193e-03  1.10151945e-02  1.16324676e-02  4.70592342e-02
  5.32041416e-02  2.

In [9]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

# demonstration of the retriever finding the relevant document chunks
questions = [
    "What novel techniques did the 'Attention is all you need' paper introduce?",
    "List the metrics were used to compare GloVe vectors with other embedding methods such as Word2Vec?"
]

for question in questions:
    print(f'QUESTION: {question}' + '\n')

    retrieved_chunks = retrieval_system.invoke(question)
    print('RETRIEVED CHUNKS: ')
    for chunk in retrieved_chunks:
        print(chunk, '\n')

    print('\n\n')

QUESTION: What novel techniques did the 'Attention is all you need' paper introduce?

RETRIEVED CHUNKS: 
page_content='2020. URL https://arxiv.org/abs/2004.14366 .
[58] Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez,
Ł ukasz Kaiser, and Illia Polosukhin. Attention is all you need. In I. Guyon, U. V . Luxburg,
S. Bengio, H. Wallach, R. Fergus, S. Vishwanathan, and R. Garnett, editors, Advances in Neural
Information Processing Systems 30 , pages 5998–6008. Curran Associates, Inc., 2017. URL
http://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf .
[59] Ashwin Vijayakumar, Michael Cogswell, Ramprasaath Selvaraju, Qing Sun, Stefan Lee, David
Crandall, and Dhruv Batra. Diverse beam search for improved description of complex scenes.' metadata={'source': 'data/papers/rag.pdf', 'page': 14} 

page_content='networks , vol. 3361, no. 10, p. 1995, 1995.
[18] A. Vaswani, N. Shazeer, N. Parmar, J. Uszkoreit, L. Jones, A. N.
Gomez, Ł. Kaiser, and I. Polo

## 7.2 : Implementing the RAG Chain [6.5pts]

Building applications with Large Language Models (LLM's) requires permissions from the LLM provider via an API key. There are two types of LLM models available for use: 
<ol>
    <li>Open-source models, which are usually smaller models with lesser capabilities (e.g. Llama created by FaceBook, Flan-T5 created by Google)</li>
    <li>Proprietary models, which are usually larger with better performace (e.g. GPT-4o by OpenAI, Gemini-1.5 by Google etc.), but aren't free to use.</li>
</ol>

In this homework, you will use an open-source HuggingFace LLM, which is free to use. In order to use a HuggingFace LLM, there are some steps that you first need to complete:
<ul>
    <li>Create a <a href= "https://huggingface.co/">Huggingface</a> Account</li>
    <li>Create a new <a href='https://huggingface.co/settings/tokens'>API Access Token</a> with the <strong>WRITE</strong> Token Type. <strong>Save this access token in a secure place and do not share it with others.</strong> For this assignment you will <strong>save the token to the READ_ME file and follow the instructions to create a <a href="https://dev.to/jakewitcher/using-env-files-for-environment-variables-in-python-applications-55a1">.env file</a> file.</strong> </li>
    <li>Accept the terms and conditions. This step may be required for certain models like <a href= 'https://huggingface.co/mistralai/Mistral-7B-v0.1'>mistralai/Mistral-7B-v0.1</a>. You can view the status of your model request <a href='https://huggingface.co/settings/gated-repos'>here</a></li>
</ul>

We also need to host our model on HuggingFace Spaces. To do this: 
<ul>
  <li>Go to: <a href= https://huggingface.co/spaces>Spaces</a></li>
  <li>Click "+ New Space" in the top right</li>
  <li>Fill in: 
    <ul>
      <li>Space name: Name of your choice</li>
      <li>SDK: Choose Gradio (blank template)</li>
      <li>Space hardware: Choose CPU basic (free)</li>
      <li>Visibility: Private</li>
    </ul>
  </li>
  <li>Click: Create Space</li>
  <li>Select Files in the top right which will take you to a new page</li>
  <li>Select + Contribute in the top right</li>
  <li>Select Upload files</li>
  <li>Add the files in hf_spaces folder provided to your space, modifying them appropriately for your models</li>
</ul>

For this section you will need to modify both the `READ_ME` and `rag_chain.py`files. 

In the `READ_ME` file, complete the following:
* Set `HUGGINGFACE_API_KEY`: Obtain your Huggingface API key and follow the instructions in the READ_ME file
* Set `GRADIO_SPACE_NAME`: Set your Gradio space name after you have created your space

### 7.2.0 : Choosing an LLM and Initalizing the Retriever System [No Points]

In the **rag_chain.py** file complete the following functions:
* **\_\_init\_\_**: This function is given to you and uses `google/flan-t5-small` as the default model. We have found that Google's Flan Models and some of Meta's Llama models work well for this question, however, you are free to play around with other models.
* **init_retriever_system**: No points are associated for implementing this function, but it is an integral component to the RAG Chain.

In [ ]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

from rag_chain import RAG_Chain
from local_tests.retriever_test import Retriever_Test
from langchain_core.vectorstores.base import VectorStoreRetriever

# Instantiate + configure the RAG_Chain class to use a HF-hosted Gradio Space via a custom LLM wrapper
rag_chain = RAG_Chain(data_dir='./data/papers/', llm_type="gradio_flan")
local_test = Retriever_Test()

# Printing the LLM used
print(rag_chain.llm) 

# Local test for RAG retriever system

# Check that retriever is of expected type:
rag_retriever = rag_chain.retriever_system
type_test = isinstance(rag_retriever, VectorStoreRetriever)
print('Your retriever is of type VectorStoreRetriever:', type_test)

# Check that the retriever retrieves relevant chunks
output_retrieved_chunks = rag_retriever.invoke(local_test.relevance_prompt)
relevance_test = True
for chunk in output_retrieved_chunks:
    if local_test.relevant_pdf not in chunk.metadata['source']:
        relevance_test = False
print('Your retriever returns chunks from the expected relevant documents:', relevance_test)

### 7.2.1 : Formatting the Prompt with PromptTemplates [2pts]

Prompts are a set of instructions that are given to an LLM in order to guide it to produce responses that are coherent, contextual and relevant. It usually takes several edits and changes to the prompt until the LLM produces the desirable response. This process is called Prompt Engineering, and you will get some experience with it here. Once you have obtained a prompt that you like, it's common practice to save it as a template for any time you want to use this prompt again. This is done with PromptTemplates.

In the **rag_chain.py** file complete the following function:

* **createPrompt**

<br>
This function takes as an input parameter, a dictionary that stores a question along with 4 possible answer choices. For example, here is what the input parameter could look like: 

{ <br>
   &emsp; 'question': "What is the main contribution of the Transformer architecture?", <br>
   &emsp; 'A': "It introduces convolutional layers for sequence tasks.", <br>
   &emsp; 'B': "It improves word embeddings using context.", <br>
   &emsp; 'C': "It removes recurrence and uses self-attention mechanisms.", <br>
   &emsp; 'D': "It uses RNNs for language modeling." <br>
}

In [ ]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

# printing the empty prompt template 
prompt_template = rag_chain.createPrompt(question={'question': "", "A": "", "B": "", "C": "", "D": ""})

print(prompt_template)

In [ ]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

# printing the formatted prompt with the question. This is what will be passed to the LLM.
from local_tests.rag_test import RAG_Test
tests = RAG_Test()

prompt_with_question = rag_chain.createPrompt(question=tests.question1)
print(prompt_with_question)

### 7.2.2 Creating Chains [4.5pts]

In LangChain, "chains" are a core concept designed to manage and streamline interactions with language models. They allow you to create sequences of operations where the output of one step can be used as the input for the next. This is particularly useful for building complex workflows and applications that involve multiple stages of processing.

Langchain offers a RetrievalQA chain, which combines the retriever module with a QA chain (short for Question-Answering). The retriever is used to retrieve relevant documents from the vectorstore, and the QA chain answers questions based on the retrieved documents.

In the **rag_chain.py** file complete the following function:

* **createRAGChain**

In [14]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

qa_chain = rag_chain.createRAGChain()

In [ ]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

from local_tests.rag_test import RAG_Test

tests = RAG_Test()

print('Local Tests for end-to-end RAG pipeline', '\n\n')

questions = tests.local_test_questions
answers = tests.local_test_answers

correct = 0
total = len(questions)
for i in range(len(questions)):
    # create the prompt with a question
    prompt_with_question = rag_chain.createPrompt(question=questions[i])
    print(prompt_with_question)

    # query the LLM
    response = qa_chain(prompt_with_question)

    print('Answer selected by the LLM:', response['result'])
    print('Correct answer:', answers[i])

    if response['result'] == answers[i]:
        correct += 1

print(f'{correct}/{total} questions answered correctly')

# Q8: Hosting and Deploying LLM and RAG [5.5pts Bonus Extra Credit]

In this question, you'll engage in a hands-on exercise exploring Ollama-hosted and Flask Ollama-hosted LLMs within the RAG pipeline you created in the previous question. You'll then deploy the RAG system using a Flask container, simulating a real-world LLM Pipeline deployment.

This exercise provides an opportunity to compare different hosting methods:

- **Cloud-based LLMs** (Hugging Face Hub or Spaces) offload the burden of compute but require external API calls.
- **Local LLMs** (Ollama) offer greater control over data, cost, and customizations, but compute is limited by our hardware.
- **Network-hosted LLMs** (Flask Ollama) allow for remote access within a private infrastructure, which is useful for on-premise deployments in industries like healthcare and finance where data privacy is critical.
  
In many real-world applications, LLMs are accessed through APIs rather than used directly. In Question 7, you worked with Hugging Face’s API via Hugging Face Spaces. Now, you’ll deploy the same RAG pipeline but using Ollama and Flask.

While we provide structured code scaffolding, we encourage you to examine the implementation carefully to understand how the various components interact. We hope this introduction gives you the necessary tools to experiment with different deployment strategies. 

**IMPORTANT: This question may need to be completed locally, it is not compatible with Google Colab. Using a GPU may also output different results than expected.**

## 8.1 : Using Ollama with RAG [No Points]

In this section we will get started with Ollama and use the Ollama LLM in the RAG pipeline.

### 8.1.1 Getting Started with Ollama

Download [Ollama](https://ollama.com/) and pull the following model:
- `llama3.2`
  
You may find the [ollama-python Documentation](https://github.com/ollama/ollama-python) (prerequisites section) and [Quickstart Guide](https://github.com/ollama/ollama/blob/main/README.md#quickstart) helpful. Make sure the ollama server is running on your machine prior to running the local test below.

**You will need to keep ollama running to complete the next sections, do not terminate it.**

In [20]:
%env TOKENIZERS_PARALLELISM=(true | false)

env: TOKENIZERS_PARALLELISM=(true | false)


In [21]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

import ta_pipeline as tap
from local_tests.q8_test import Q8_Test

local_test = Q8_Test()

# Check that Ollama returns the expected response
response = tap.query_ollama(local_test.ollama_query)
response_check = response == local_test.ollama_response
print('Your Ollama server returned the expected response:', response_check)

Your Ollama server returned the expected response: True


### 8.1.2 Using Ollama with RAG

In the **rag_chain.py** file complete the following function:
* **set_ollama_only**: Initialize the LLM using the Ollama wrapper (see [documentation](https://api.python.langchain.com/en/latest/llms/langchain_community.llms.ollama.Ollama.html)) and set the model. 

In [22]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

from local_tests.q8_test import Q8_Test
from rag_chain import RAG_Chain

# Local test for RAG using Ollama LLM

# Check that Ollama works with rag_chain.py
rag_chain_oo = RAG_Chain(data_dir='./data/papers/', llm_type="ollama_only", init_retriever=False)
rag_chain_oo.llm.temperature = 0
local_test = Q8_Test()

# Printing the LLM used
print(f"LLM Info:\n {rag_chain_oo.llm}\n") 

# Check that RAG returns the expected response
response = rag_chain_oo.query_the_llm(local_test.ollama_rag_query)
response_check = response == local_test.ollama_rag_response
print('Your Ollama LLM in the RAG system returned the expected response:', response_check)
print("\nQUERY:", local_test.ollama_rag_query)
print("EXPECTED RESPONSE:", local_test.ollama_rag_response)
print(f"Your Response: {response}" if not response_check else "")

LLM Info:
 Ollama
Params: {'model': 'llama3.2', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': 0, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}

Your Ollama LLM in the RAG system returned the expected response: True

QUERY: True or False, a bird is a bat.
EXPECTED RESPONSE: False. Birds and bats are two distinct groups of animals that belong to different classes (Aves for birds and Chiroptera for bats) and have many physical and behavioral differences.



## 8.2 : Deploying an LLM and using it with RAG [No Points]

Next, we will deploy the Ollama LLM from 8.1.1 by containerizing the LLM using Flask. While Ollama works locally, Flask can expose the LLM for network use. We will query into this Flask Ollama LLM by reusing Langchain's OpenAI wrapper. 

### 8.2.1: Using Flask to Containerize the Ollama LLM

In the **llm_app.py** file complete the following function:
* **generate**: Extract the data from the Flask message request, complete the request to the Ollama server using the extracted data, format the response as required per the OpenAI wrapper and return.

In your terminal or command prompt, run the llm_app.py file before running the local test cell below.

**You will need to keep llm_app.py running to complete the next section, do not terminate it.**

In [23]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

import ta_pipeline as tap
from local_tests.q8_test import Q8_Test

local_test = Q8_Test()

# Local test for Flask Ollama server

#Check that Ollama returns the expected response
response = tap.query_flask_ollama(local_test.ollama_query)
response_check = response == local_test.ollama_response
print('Your Flask Ollama server returned the expected response:', response_check)

Your Flask Ollama server returned the expected response: True


### 8.2.2 Using Flask Ollama with RAG

In the **rag_chain.py** file complete the following function:
* **set_flask_ollama**: Initialize the LLM using the OpenAI wrapper and set the model.

In [24]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

import ta_pipeline as tap
from local_tests.q8_test import Q8_Test
from rag_chain import RAG_Chain

local_test = Q8_Test()

# Local test for RAG using Flask Ollama

# Check that Ollama works with rag_chain.py
rag_chain_fo = RAG_Chain(data_dir='./data/papers/', llm_type="flask_ollama", init_retriever=False)
rag_chain_fo.llm.temperature = 0
local_test = Q8_Test()

# Printing the LLM used
print(f"LLM Info:\n {rag_chain_fo.llm}\n") 

# Check that RAG returns the expected response
response = rag_chain_fo.query_the_llm(local_test.flask_ollama_rag_query)
response_check = response == local_test.flask_ollama_rag_response
print('Your Flask Ollama LLM in the RAG system returned the expected response:', response_check)
print("\nQUERY:", local_test.flask_ollama_rag_query)
print("EXPECTED RESPONSE:", local_test.flask_ollama_rag_response)
print(f"YOUR RESPONSE: {response}" if not response_check else "")

LLM Info:
 OpenAI
Params: {'model_name': 'llama3.2', 'temperature': 0, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'seed': None, 'logprobs': None, 'max_tokens': 256}

Your Flask Ollama LLM in the RAG system returned the expected response: True

QUERY: True or False, a bird is a bat.
EXPECTED RESPONSE: False. Birds and bats are two distinct groups of animals that belong to different classes (Aves for birds and Chiroptera for bats) and have many physical and behavioral differences.



## 8.3 : Deploying RAG using Flask [5.5pts]

Now we are ready to deploy the RAG implementation from 8.2 using Flask. 

### 8.3.1 Deploying RAG Local Test [No Points]

In the **rag_app.py** file complete the following function:
* **query_rag**: Extract the data from the message request, update the LLM parameters based on the extracted information, query the RAG, format the response and return.

In your terminal or command prompt, run the rag_app.py file before running the local test cell below.

**You will need to keep rag_app.py running to complete the final TA pipeline test, do not terminate it. You should have Ollama and llm_app.py running before rag_app.py.**

In [25]:
###############################
### DO NOT CHANGE THIS CELL ###
###############################

import ta_pipeline as tap
from local_tests.q8_test import Q8_Test

local_test = Q8_Test()

# Local test for Ollama server

# Check that flask rag returns the expected response
response = tap.query_flask_rag(local_test.flask_rag_query)
response_check = response[0]["text"] == local_test.flask_rag_response
print('Your Flask RAG system returned the expected response:', response_check)
print("\nQUERY:", local_test.flask_rag_query)
print("EXPECTED RESPONSE:", local_test.flask_rag_response)
print(f"YOUR RESPONSE: {response[0]['text']}" if not response_check else "")

Your Flask RAG system returned the expected response: True

QUERY: Does Glove use word similarity?
EXPECTED RESPONSE: Yes, according to the text, GloVe uses word-word co-occurrence counts, which implies that it does use word similarity as a basis for generating word representations.



### 8.3.2 Deploying RAG Final Test [5.5 pts Bonus Extra Credit]

Once you are able to pass the local test in 8.3.1, proceed to running the test below for the final test of your RAG system. Submit the generated hashed_answers.json file to Gradescope. 

In [26]:
%%time
###############################
### DO NOT CHANGE THIS CELL ###
###############################

import ta_pipeline as tap

tap.main()

#### 🐙 <font color='darkred'>Submit hashed_answers.json to Gradescope</font>
**Running the TA pipeline should produce hashed_answers.json. Submit this file to Gradescope along with llm_app.py, rag_app.py, rag_chain.py, and retriever.py.**

**Congrats, you have reached the end of Homework 4 🥳**